# Magnetic Interactive
This calculation is to calculate how the mouse cursor with respond to raycaster on a vertices an manipulate each vertex in vertex shader. The idea is to use [Gradient Scalar Formula](https://en.wikipedia.org/wiki/Gradient) . First, imagine the number of vertices on a plane is like this:

In [ ]:
from sage.repl.ipython_kernel.widgets_sagenb import slider
from sage.all import *

@interact
def see_plot(
        x_vertex=slider(default=10, vmin=0, vmax=100, label='Total X Vertices'),
        z_vertex=slider(default=10, vmin=0, vmax=100, label='Total Z Vertices'),
        space_between=slider(default=2, vmin=0, vmax=100),
        vertex_size=slider(default=0.2, vmin=0.0, vmax=1.0)
):
    # Build bunch of spheres to imitate vertex positon in a ThreeJS Plane Geometry Format
    position = vector([0. - float((x_vertex * space_between)/2), 0., 0.]) # Start Position (Center from X Axis between negative and positive)
    vertices = [] # Collect Sphere as Vertices

    scene = sphere(vector([0.,0.,0.]), opacity=0.5, size=0.0)

    for x in range(x_vertex):
        for z in range(z_vertex):
            vertex = sphere(position + vector([x * space_between, 0., z * space_between]), size=vertex_size * space_between) # Make sphere size uniform to space_between value so that the sphere always appear the same size in viewer no matter how big does the space between is.
            vertices.append(vertex)
            scene += vertex

    # Vertex diameter of radius^2 + total vertex (Remove 1 due to index) * (all spaces between each x vertices)
    center_x = (vertex_size**2 + (x_vertex-1) * space_between)/2
    center_z = (vertex_size**2 + (z_vertex-1) * space_between)/2
    mouse_offset = vector([center_x, -5., center_z])
    scene += sphere(position + mouse_offset, size=vertex_size * space_between, color=Color('red'))

    scene.show()

## Begin Magnetic Pull
A common way to do this (in graphics, physics, and shader code) is:

1. What we want

    You want each vertex $p \in \mathbb{R}^3$ to be pulled toward the mouse position $m \in \mathbb{R}^3$.
    That’s equivalent to a force field that decreases with distance, like gravity or magnetism.


3. General force field formula
    We can define the pull force vector as

    $\vec{F}(p) \;=\; k \cdot \frac{m - p}{\|m - p\|^n}$

    where
    - $p$ = vertex position
    - $m$ = mouse position
    - $k > 0$ = strength of attraction
    - $n \geq 1$ = falloff exponent
    - $n=1$: linear pull
    - $n=2$: inverse-square law (like gravity or magnetism)
    - $n>2$: very localized pull


3. Updated vertex position

    Each frame, you update the vertex position by adding a fraction of this force:

    $p’ = p + \alpha \, \vec{F}(p)$

    where $\alpha \in (0,1)$ is a blending factor that controls how fast vertices move toward the mouse.


4. Gradient connection

    Notice that this force field is the negative gradient of the potential function

    $U(p) = \frac{1}{\|p - m\|}$

    since

    $\nabla U(p) = - \frac{m - p}{\|m - p\|^3}$

    So the magnetic-like attraction you want is literally a gradient descent toward the mouse.

In [ ]:
from sage.all import *

def lerp(a, b, t):
    return a + (b - a) * t

def deg_to_rad(deg):
    return deg * pi / 180.0

def rad_to_deg(rad):
    return rad * 180.0 / pi

def euler_x_axis(deg):
    return matrix(
        [
            [1, 0, 0],
            [0, cos(deg_to_rad(deg)), sin(deg_to_rad(deg))],
            [0, -sin(deg_to_rad(deg)), cos(deg_to_rad(deg))]
        ]
    )

def pitch_y_axis(deg):
    return matrix(
        [
            [cos(deg_to_rad(deg)), 0,  -sin(deg_to_rad(deg))],
            [0, 1, 0],
            [sin(deg_to_rad(deg)), 0, cos(deg_to_rad(deg))]
        ]
    )

def yaw_z_axis(deg):
    return matrix(
        [
            [cos(deg_to_rad(deg)), -sin(deg_to_rad(deg)), 0],
            [sin(deg_to_rad(deg)),  cos(deg_to_rad(deg)), 0],
            [0, 0, 1]
        ]
    )

def unit_vector(v):
    return v / norm(v)

def distance(v1, v2):
    return norm(v2 - v1)

def gradient_pull(pos, mouse, threshold, strength=1.0, falloff=2, eps=1e-9):
    """
    Magnetic-like pull:
    pos      = vertex position
    mouse    = mouse position
    threshold= max influence distance
    strength = force multiplier
    falloff  = 1 (linear), 2 (inverse-square), etc.
    """
    delta = mouse - pos
    d = norm(delta)
    if d < threshold and d > eps:
        influence = strength / (d**falloff)
        # print("Original: %s Changes: %s Mouse: %s" % (pos, pos + influence * delta, mouse))
        return pos + influence * delta
    return pos

def rotate_around_point(point_to_rotate, pivot_point, rotation_matrix):
    """ Rotates a point around a given pivot. """
    # Step 1: Translate the point so the pivot is at the origin
    translated_point = point_to_rotate - pivot_point

    # Step 2: Rotate the translated point
    rotated_point = rotation_matrix * translated_point

    # Step 3: Translate the point back
    final_point = rotated_point + pivot_point

    return final_point


def magnetic_animation(num_cycles=8, steps_per_cycle=15):
    frames = []
    x_vertex = 10
    z_vertex = 10
    space_between = 2
    vertex_size = 0.5
    threshold_distance = 5
    scale_factor = 3


    # Start position: center on X axis
    position = vector([-(x_vertex * space_between) / 2.0, 0., 0.])

    # Vertex grid "center"
    center_x = (vertex_size**2 + (x_vertex - 1) * space_between) / 2
    center_z = (vertex_size**2 + (z_vertex - 1) * space_between) / 2
    mouse_pos = position + vector([center_x, 0., center_z])

    # ⭐ DEFINE THE PIVOT POINT for the rotation ⭐
    # Let's use the initial position of the red mouse sphere as our pivot.
    pivot_point = mouse_pos

    total_steps = num_cycles * steps_per_cycle
    for i in range(total_steps):
        # fresh scene
        scene = Graphics()

        # Calculate the rotation for the current frame
        # This will complete a full 360-degree rotation over the animation duration
        current_angle_deg = 360.0 * (i / total_steps)
        rotation_matrix = pitch_y_axis(current_angle_deg)

        # grid of spheres
        for x in range(x_vertex):
            for z in range(z_vertex):
                pos = position + vector([x * space_between, 0., z * space_between])
                end_pos = pos * scale_factor

                moved = gradient_pull(end_pos, mouse_pos,
                  threshold_distance,
                  strength=2.0,
                  falloff=2)

                # ✅ ROTATE the final sphere position around the pivot
                final_moved_pos = rotate_around_point(moved, pivot_point, rotation_matrix)
                scene += sphere(center=moved, size=vertex_size, color='blue')

        # ✅ ROTATE the red "mouse sphere" around the pivot as well
        final_mouse_pos = rotate_around_point(mouse_pos, vector([center_x, 0., center_z]), rotation_matrix)
        scene += sphere(center=final_mouse_pos, size=vertex_size, color='red')

        frames.append(scene)

    return frames

# ---------- run ----------
frames = magnetic_animation(num_cycles=2, steps_per_cycle=10)
A = animate(frames, axes=True).interactive(online=True)
A.show()